In [17]:
import pandas as pd

In [28]:
df = pd.read_csv('pre_processed.csv')

In [19]:
decks_by_planet = {
    'Earth':['E','F','G'],
    'Europa': ['A','B','C','D','E','T'],
    'Mars': ['D','E','F']
}

decks_by_planet_no_bills = {
    'Earth':['G'],
    'Europa':['B'],
    'Mars': ['E','F']
}

planet_by_deck = {
    'A':['Europa'],'B':['Europa'],'C':['Europa'],'D':['Europa','Mars'],'E':['Europa','Mars','Earth'],
    'F':['Earth','Mars'],'G':['Earth'],'T':['Europa']
}

homeplanets = ['Earth', 'Europa', 'Mars']

all_cabin_sides = ['P','S']

all_cabin_decks = list(df.dropna(subset = ['Deck']).Deck.unique())


In [ ]:
def fill_potential_decks(df):
    def func_potential_decks(row):
        if pd.isna(row.Cabin):
            if row.Bills == 0 and not pd.isna(row.HomePlanet):
                if len(df[df.Group == row.Group].dropna(subset = 'Deck').Deck.unique()) > 1:
                    return decks_by_planet_no_bills[row.HomePlanet]
            if not pd.isna(row.HomePlanet):
                return decks_by_planet[row.HomePlanet]
            else:
                return all_cabin_decks
    df['potential_decks'] = df.apply(func_potential_decks, axis = 1)
    return df

In [ ]:
df = fill_potential_decks(df)

In [29]:
df4 = pd.read_csv('df4.csv')

In [31]:
df4.potential_decks.value_counts()

potential_decks
['E', 'F', 'G']                   148
['A', 'B', 'C', 'D', 'E', 'T']     80
['D', 'E', 'F']                    58
['G']                              13
Name: count, dtype: int64

In [21]:
def rooms_to_fill(df):
    rooms = {}
    for deck in all_cabin_decks:
        for side in all_cabin_sides:
            rooms_seen = list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())
            largest_room_number = int(max(list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())))
            for i in range(largest_room_number):
                if i not in rooms_seen:
                    if deck not in rooms:
                        rooms[deck] = {'P':[],'S':[]}
                    rooms[deck][side].append(i)
    return rooms

                    
                    
                    
            

# there arent any free rooms for it so has to share


In [32]:
def no_free_rooms_so_shares_2(df):
    free_rooms = rooms_to_fill(df)
    for index,passenger in df[df.Cabin.isna()].iterrows():
        options = False
        for deck in passenger.potential_decks:
            for side in all_cabin_sides:
                if deck not in free_rooms:
                    continue
                if side not in free_rooms[deck]:
                    continue
                potential_room_numbers = free_rooms[deck][side]
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:]
                before_slice = before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique()
                if len(before_slice) > 1:
                    top_room_number_before = max(before_slice)
                else:
                    top_room_number_before = -2
                    
                smallest_room_number_after = min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())
                if top_room_number_before + 1 == smallest_room_number_after:
                    if int(top_room_number_before) + 1 in potential_room_numbers:
                        options = True
        if not options:
            if len(df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))].Cabin.unique()) == 1:
                print(passenger)
    return df
        
                

In [38]:
df4[df4.Cabin.isna()].potential_decks.iloc[0]

"['A', 'B', 'C', 'D', 'E', 'T']"

In [34]:
df4 = no_free_rooms_so_shares_2(df4)

TypeError: only list-like objects are allowed to be passed to isin(), you passed a `str`

In [ ]:
def no_free_rooms_so_shares(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        options = False
        for deck in passenger.potential_decks:
            for side in all_cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options = True
                        break
            if options:
                break
        if not options:
            other_group_member = df[(df.Group == passenger.Group) & (~df.Cabin.isna()) & (df.Deck.isin(passenger.potential_decks))]
            print(index)
            if len(other_group_member.Cabin.unique()) == 1:
                
                df.loc[index,'Cabin'] = other_group_member.iloc[0].Cabin
                
    df[['Deck','Number','Side']] = df.apply(fill_deck_cabin_side,axis = 1)
    return df
  
            # Slice the DataFrame first and then apply the boolean mask

df2 = no_free_rooms_so_shares(df2)